# Aufgabe Teilmodul 2.1
## Auswahl Ihrer Daten
Suchen Sie auf bibsonomy mit einem selbst gewählten Suchbegriff nach Publikationen. Auf Bibsonomy lassen sich die Ergebnisse einer Keyword-Suche (z.B. "Bibliothek") in einer Vielzahl verschiedener Formate exportieren (Limit ggf. vorher auf 1000 erhöhen) und lokal speichern. Geben Sie dafür einen Suchbegriff oben rechts in die Maske ein und bestätigen Sie die Eingabe. Rechts neben der Überschrift der rechten Spalte ("publications"/"Publikationen") klicken Sie ganz rechte auf das Symbol mit dem Pfeil nach rechts, der aus einem Kasten zeigt ("export options for displayed posts"/"Exportieren für angezeigte Einträge"), dann auf mehr/more. Wählen Sie ZUERST bei posts/Einträge "1000" aus, und wählen sie dann in dem Format drop-Down-Menü fast ganz unten JSON aus. Es öffnet sich die JSON Datei in ihrem Browser.  Sie können die URL wie im Tutorial über das requests Modul in Python öffnen.

Alternativ können Sie die Datei auch lokal speichern. Dafür rechtsklicken Sie irgendwo, wählen Sie "save as..."/"speichern unter..." und speichern Sie die Datei unter einem passenden Dateinamen mit der Endung ".json".

Für eine Nutzung im zweiten Modulteil ist ein großer Datensatz sinnvoll. Fügen Sie daher die Ergebnisse mehrerer Suchen mit Pandas in einem Datensatz zusammenführen.
Fügen sie ihren Datensätzen dabei den Suchterm hinzu, der zu den Ergebnissen geführt hat. Tipp: Informationen dazu finden Sie unter "Daten Verändern und Hinzufügen".

Nutzen sie dabei Python (vorzugsweise in einem Jupyter Notebook) mithilfe von Pandas. Sollten Sie Probleme haben, schauen Sie im Pandas Tutorial unter "JSON einlesen" oder bitten Sie Fabian Haak im Moodle Austauschforum um Hilfe.
Legen Sie Ihr Notebook bzw. Python-Script mit den Lösungen in einem "Modul_2-1" Ordner im eigenen Repositorium auf GitHub ab. Fügen Sie mich als zusätzlichen Collaborator (GitHub-Name: HaakFab) hinzu, so kann ich besser Feedback oder Hilfestellungen geben.

## Exportformat erstellen
Diese Aufgabe dient als direkte Vorbereitung des nächsten Modulteils. Wir empfehlen, Pandas für die Bearbeitung der Aufgabe zu nutzen. 

### Datenaufbereitung
Für den nächsten Teil dieses Modules möchten wir unseren Datensatz wieder als JSON-File ablegen. Zunächst möchten wir aber die Bibsonomy-Daten noch etwas zu bereinigen:
- Filtern Sie die Daten so, dass nur noch Einträge des Typs "Publication" enthalten sind.
- Überlegen Sie, welche Felder Sie möglicherweise umbenennen wollen, und welche Sie nicht benötigen. Benennen Sie entsprechend um und wählen Sie nur die Spalten, die für Sie sinnvolle Informationen enthalten. Mögliche Änderungen der Daten sind das Umbenennen von Felder zu eindeutigeren Bezeichnungen (wie beispielsweise im Pandas Tutorial "label" zu "title"). Filtern Sie ihre Daten jedoch nicht zu drastisch und entfernen Sie nicht zu viele Felder, damit im zweiten Modulteil noch genügend Informationen übrig bleiben.
- Wenn sie mehrere Suchergebnisse zusammengeführt haben, ist es sinnvoll, Nachduplikaten zu überprüfen.
- BONUS: Wie bei den Jareszahlen ist es durchaus sinnvoll, zu überprüfen, ob alle Einträge "sauber" vorliegen. Stellen sie dafür sicher, dass alle Einträge Informationen zu Titel, AutorInnen und Veröffentlichungsjahr enthalten und dass das Jahr eine sinnvolle Zahl ist!
Daten Export

### Erstellen Sie einen sauberen Export im JSON-Format, das von Solr akzeptiert wird.

Für die Weiterverwendung ist nicht relevant, wie die einzelnen Felder benannt sind oder wie viele Felder es gibt, Es ist aber entscheidend, dass die äußere Form einer Liste von Dictionaries gewahrt wird. (HINWEIS BEI PROBLEMEN: Schauen Sie im Pandas Tutorial unter "Export als JSON")

In [2]:
import pandas as pd
import requests

# zum Zusammenbauen der Bibsonomy URL brauchen wir alles, was vor und nach der Query kommt
url_base = "https://www.bibsonomy.org/json/search/"
url_attr = "?items=1000"

def df_from_query(query):
    # die URL; wir müssen Leerzeichen ersetzen mit "%20" nach ASCII Encoding guideline
    url = url_base + query.replace(" ", "%20") + url_attr
    
    result = requests.get(url)
    json_data = result.json()
    
    # im Feld "items" stecken die relevanten Dateneinträge
    dataframe = pd.DataFrame(json_data["items"])

    # wir filtern so, dass nur Publikationen bleiben
    dataframe.loc[dataframe["type"] == "Publication"]

    # wir möchten die query in die Daten eintragen 
    dataframe["query"] = query
    
    # die Rückgabe der Methode nicht vergessen:
    return dataframe

# wir erstellen eine leere Liste, in die wir unsere Dataframes packen
dataframes = []

# eine Liste von Suchanfragen
queries = ["artificial intelligence", "AI", "machine learning", "language model", "KI", "künstliche intelligenz", "maschinelles lernen", "sprachmodell"]

# jetzt gehen wir die Liste von Suchanfragen durch und rufen unsere Funktion für jede Suchanfrage auf
for query in queries:
    df = df_from_query(query)
    # schließlich fügen wir das DataFrame über append unserer Liste von Dataframes hinzu
    dataframes.append(df)

    # wie wir uns eine Statusausgabe schreiben würden:
    # print("Ergebnisse für ", query , " gefunden. Dataframe enthält ",len(df), " Einträge.")
    
    # mit einem f String lassen sich Variablen in Strings viel einfacher formatieren:
    print(f"{len(df)} Ergebnisse für query {query} gefunden.")

2000 Ergebnisse für query artificial intelligence gefunden.
2000 Ergebnisse für query AI gefunden.
2000 Ergebnisse für query machine learning gefunden.
1311 Ergebnisse für query language model gefunden.
1410 Ergebnisse für query KI gefunden.
1275 Ergebnisse für query künstliche intelligenz gefunden.
295 Ergebnisse für query maschinelles lernen gefunden.
30 Ergebnisse für query sprachmodell gefunden.


In [3]:
# Da die dataframes die selbe From (column titles sind identisch, wenn für alle vorhanden) haben, können wir die Dataframes über concat zusammenfügen
data_conc = pd.concat(dataframes, ignore_index=True)

#### Filtern Sie die Daten so, dass nur noch Einträge des Typs "Publication" enthalten sind.

In [4]:
data = data_conc.loc[data_conc["type"] == "Publication"] #war vorher noch nicht gespeichter, nur als Ansicht (kein " = ")

#### Überlegen Sie, welche Felder Sie möglicherweise umbenennen wollen, und welche Sie nicht benötigen. 


In [5]:
data.columns.to_list()

['type',
 'id',
 'tags',
 'intraHash',
 'label',
 'user',
 'description',
 'date',
 'changeDate',
 'count',
 'url',
 'interHash',
 'pub-type',
 'journal',
 'year',
 'author',
 'authors',
 'editor',
 'editors',
 'volume',
 'number',
 'pages',
 'abstract',
 'ee',
 'language',
 'issn',
 'bibtexKey',
 'publisher',
 'address',
 'isbn',
 'file',
 'urldate',
 'series',
 'refid',
 'note',
 'doi',
 'shorttitle',
 'booktitle',
 'location',
 'ean',
 'asin',
 'dewey',
 'numpages',
 'issue_date',
 'acmid',
 'owner',
 'pmid',
 'timestamp',
 'username',
 'intrahash',
 'added-at',
 'numer',
 'interhash',
 'biburl',
 'groups',
 'notes',
 'size',
 'citeulike-article-id',
 'priority',
 'comment',
 'book',
 'date-modified',
 '_originaltype',
 '_keywords',
 '_originalcomment',
 'eventtitle',
 'eventdate',
 'googleid',
 'cluster',
 'citeseer-isreferencedby',
 'rights',
 'oai',
 'serial',
 'articleno',
 'pagetotal',
 'ppn_gvk',
 'subtitle',
 'local-url',
 'oldnote',
 'posted-at',
 'date-added',
 'uri',
 'pii

In [22]:
data_sel = data[[
    "id",
    "label",
    "tags",
    "url",
    "authors",
    "year"
]]

In [23]:
data_sel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6219 entries, 1000 to 10320
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       6219 non-null   object
 1   label    6219 non-null   object
 2   tags     6219 non-null   object
 3   url      6219 non-null   object
 4   authors  5928 non-null   object
 5   year     6219 non-null   object
dtypes: object(6)
memory usage: 340.1+ KB


#### Benennen Sie entsprechend um und wählen Sie nur die Spalten, die für Sie sinnvolle Informationen enthalten. Mögliche Änderungen der Daten sind das Umbenennen von Felder zu eindeutigeren Bezeichnungen (wie beispielsweise im Pandas Tutorial "label" zu "title"). 

In [24]:
data_sel = data_sel.rename(columns={"label":"title"})

#### Wenn sie mehrere Suchergebnisse zusammengeführt haben, ist es sinnvoll, nach Duplikaten zu überprüfen.

In [29]:
data_sel_dup = data_sel.drop_duplicates(subset="title", keep="last")

#### SABINE:
- Sollte man Dublikate vor dem Merge entfernen lassen oder lieber danach?

#### BONUS: Wie bei den Jareszahlen ist es durchaus sinnvoll, zu überprüfen, ob alle Einträge "sauber" vorliegen. Stellen sie dafür sicher, dass alle Einträge Informationen zu Titel, AutorInnen und Veröffentlichungsjahr enthalten und dass das Jahr eine sinnvolle Zahl ist!

In [31]:
data_clean = data_sel_dup.dropna(subset=["title", "authors", "year"])

### Wenn noch Zeit ist (sonst: wird nachgereicht!)

#### LARA:
- habe bei der Autoren-Spalte alle NaN Einträge rausgelöscht später aber festgestellt, dass ich noch solche Einträge habe: [?]
Wie könnte ich diese rausfiltern und löschen sodass ich am Ende nur Namen in der Spalte habe?
- in der Spalte "language" hatte ich Titel auf deutsch und englisch. Deutsche Titel hatten bspws. folgende Werte: dt, ger, german
Wie kann ich diese Werte alle in "deutsch" umbenennen?



1. Language Spalte säubern --> nur noch EN oder DE als Werte! Bei keiner angabe EN eintragen


## PAUSE

# Gruppenaufgabe 1
### Aufgabenbeschreibung
Wir möchten nun zusätzlich zu den Bibsonomy Daten auch noch Daten aus einer weiteren Quelle zusammenführen, in diesem Fall Semantic Scholar.

1. Lade "bibsonomy.json" und "semantic_scholar.json" in Pandas als ein Dataframe, füge eine Spalte "source" hinzu, in der du festhälst, woher die Suchergebnisse kommen  (also: bibsonomy oder semantic_scholar).
2. Selektiere und benenne die Spalten so um, dass aus beiden Quellen Titel, Autoren, URL, Abstract, Keywords, Query (Suchanfrage) sowie das Jahr enthalten und gleich benannt sind.
3. Stelle sicher, dass die Autoren gleich als Liste formatiert sind (also z. B. [Max Mustermann, Lieschen Müller] oder [Mustermann M., Müller L.]).
4. Führe die Datensätze zusammen.
5. Stelle sicher, dass alle Einträge für "Jahr" eine 4-stellige Zahl (int) sind.
6. Entferne Duplikate (anhand der Spalte Title).
7. Sorge dafür, dass es einen durchgängigen Index gibt.